In [1]:
! git clone https://github.com/tonythomasndm/CSE508_Winter2024_A1_Dataset.git
! mkdir preprocessed_files

fatal: destination path 'CSE508_Winter2024_A1_Dataset' already exists and is not an empty directory.
A subdirectory or file preprocessed_files already exists.


In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

#Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

additional_path=""
# for google collab use -/content/
#Looping through the dataset
for i in range(1,1000):
  num= "% s" % i
  read_path=additional_path+'CSE508_Winter2024_A1_Dataset/text_files/file' + num + '.txt'
  file_read=open(read_path,'r')
  text=file_read.read()

  # Lower the text
  text = text.lower()

  # Tokenize the text
  tokens = word_tokenize(text)

  # Remove stopwords
  stop_words = set(stopwords.words('english'))
  tokens = [token for token in tokens if token not in stop_words]

  # Remove punctuations
  tokens = [token for token in tokens if token not in string.punctuation]

  # Remove blank space tokens
  tokens = [token for token in tokens if token.strip()]

  sentence=""
  for token in tokens:
    sentence=sentence+token+' '

  #Saving the Preprocessed Files
  file_write=open(additional_path+'preprocessed_files/file'+ num +'.txt','w')
  file_write.write(sentence)
  file_read.close()
  file_write.close()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ttony\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ttony\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Printing Sample 5 random documents before and after preprocessing
import random

# Generate 5 random numbers between 1 and 999
random_numbers = random.sample(range(1, 1000), 5)

for i in random_numbers:
  num= "% s" % i
  read_path=additional_path+'CSE508_Winter2024_A1_Dataset/text_files/file' + num + '.txt'
  file_read=open(read_path,'r')
  text=file_read.read()
  print("\nDoc "+num+" before preprocessing")
  print(text)
  file_read.close()
  file_read=open(additional_path+'preprocessed_files/file'+ num +'.txt','r')
  text=file_read.read()
  print("\nDoc "+num+" after preprocessing")
  print(text)
  file_read.close()
  print("-"*170)



Doc 958 before preprocessing
Suit very well if you have a mixer thats a desk top model with out any way of rack mounting it.

Doc 958 after preprocessing
suit well mixer thats desk top model way rack mounting 
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Doc 175 before preprocessing
My ES*335 fit loose and needed some padding added to keep the guitar snug, but if you don't mind doing that it's a good deal...

Doc 175 after preprocessing
es 335 fit loose needed padding added keep guitar snug n't mind 's good deal ... 
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Doc 291 before preprocessing
Highly recommend. It did wonders for my Kremona Solea. It's like a totally different guitar. Much more lively.

Doc 291 after preprocessing
highly r

In [4]:
#Creating unigram inverted index
inverted_index={}
total_docs=set()
for i in range(1,1000):
  num= "% s" % i
  file_read=open(additional_path+'preprocessed_files/file'+ num +'.txt','r')
  text=file_read.read()
  tokens=text.split()
  doc_id="file"+num+".txt"
  total_docs.add(doc_id)
  for token in tokens:
    if token not in inverted_index:
        inverted_index[token] = set()
    inverted_index[token].add(doc_id)

#Storing the Inverted Index
import pickle
f=open(additional_path+"inverted_index.txt","wb")
pickle.dump(inverted_index,f)


In [5]:
#Loading the inverted Index
f=open(additional_path+"inverted_index.txt",'rb')
inverted_index = pickle.load(f)

def andOp(docs1,docs2):
    return docs1.intersection(docs2)

def orOp(docs1,docs2):
    return docs1.union(docs2)

def andNotOp(docs1,docs2,total_docs):
    not_docs2 = total_docs.difference(docs2)
    return docs1.intersection(not_docs2)

def orNotOp(docs1,docs2,total_docs):
    not_docs2 = total_docs.difference(docs2)
    return docs1.union(not_docs2)

def queryTextPreProcess(text):
  # Lower the text
  text = text.lower()

  # Tokenize the text
  tokens = word_tokenize(text)

  # Remove stopwords
  stop_words = set(stopwords.words('english'))
  tokens = [token for token in tokens if token not in stop_words]

  # Remove punctuations
  tokens = [token for token in tokens if token not in string.punctuation]

  # Remove blank space tokens
  tokens = [token for token in tokens if token.strip()]

  return tokens

tokensInp=[]
operatorsInp=[]

#Taking Queries
n=int(input())
for i in range(0,n):
  queryText=input()
  operators=input().lower().split(",")
  operatorsInp.append(operators)
  tokens=queryTextPreProcess(queryText)
  tokensInp.append(tokens)

#Processing Queries
for i in range(0,n):

  print("\nQuery",(i+1),":", end=" ")
  operators=operatorsInp[i]
  tokens=tokensInp[i]
  query=tokens[0]
  op_count=0
  docs=inverted_index[tokens[0]]
  q=len(tokens)

  for iQ in range(1,q):
      op=operators[op_count].strip()
      query=query+' '+op.upper()
      term=tokens[iQ].strip()
      op_count+=1
      if term not in inverted_index:
          inverted_index[term]=set()
      if op == 'and':
        docs=andOp(docs,inverted_index[term])
      elif op =='or':
        docs=orOp(docs,inverted_index[term])
      elif op =='and not':
        docs=andNotOp(docs,inverted_index[term],total_docs)
      elif op=='or not':
        docs=orNotOp(docs,inverted_index[term],total_docs)
      query=query+' '+term
  print(query)
  print("Number of documents retrieved :", len(docs))
  if len(docs):
    print("Names of the documents retrieved : ",end="")
    for i in docs:
      print(i,end=", ")
  else:
    print("Names of the documents retrieved: None")



Query 1 : live AND music
Number of documents retrieved : 1
Names of the documents retrieved : file22.txt, 
Query 2 : works AND NOT nicely
Number of documents retrieved : 95
Names of the documents retrieved : file45.txt, file703.txt, file393.txt, file773.txt, file354.txt, file645.txt, file620.txt, file356.txt, file347.txt, file285.txt, file780.txt, file30.txt, file130.txt, file173.txt, file597.txt, file448.txt, file823.txt, file487.txt, file712.txt, file317.txt, file332.txt, file968.txt, file377.txt, file686.txt, file878.txt, file605.txt, file631.txt, file724.txt, file428.txt, file568.txt, file254.txt, file50.txt, file408.txt, file184.txt, file276.txt, file897.txt, file322.txt, file2.txt, file330.txt, file390.txt, file44.txt, file261.txt, file417.txt, file945.txt, file693.txt, file580.txt, file147.txt, file353.txt, file267.txt, file344.txt, file647.txt, file90.txt, file161.txt, file684.txt, file857.txt, file293.txt, file595.txt, file600.txt, file810.txt, file368.txt, file874.txt, file8

In [6]:
#Creating positional index
positional_index={}
total_docs=set()
for i in range(1,1000):
  num= "% s" % i
  file_read=open(additional_path+'preprocessed_files/file'+ num +'.txt','r')
  wordPosition=1
  text=file_read.read()
  tokens=text.split()
  doc_id="file"+num+".txt"
  total_docs.add(doc_id)
  for token in tokens:
    if token not in positional_index:
        positional_index[token] ={}
        positional_index[token]={doc_id:set()}
    elif doc_id not in positional_index[token]:
        positional_index[token][doc_id] = set()
    positional_index[token][doc_id].add(wordPosition)
    wordPosition+=1

#Storing the Positional Index
import pickle
f=open(additional_path+"positional_index.txt","wb")
pickle.dump(positional_index,f)

In [7]:
#Loading the positional Index
f=open(additional_path+"positional_index.txt",'rb')
inverted_index = pickle.load(f)

def queryTextPreProcess(text):
  # Lower the text
  text = text.lower()

  # Tokenize the text
  tokens = word_tokenize(text)

  # Remove stopwords
  stop_words = set(stopwords.words('english'))
  tokens = [token for token in tokens if token not in stop_words]

  # Remove punctuations
  tokens = [token for token in tokens if token not in string.punctuation]

  # Remove blank space tokens
  tokens = [token for token in tokens if token.strip()]

  return tokens

tokensInp=[]
operatorsInp=[]

#Taking Queries
n=int(input())
for i in range(0,n):
  queryText=input()
  tokens=queryTextPreProcess(queryText)
  tokensInp.append(tokens)

#Recursive Function
docs=set()
def fn(index,tokens,currentWordPos,currentDoc):
  if(index==len(tokens)):
    docs.add(currentDoc)
    return
  else:
    if tokens[index] not in positional_index.keys():
      return
    if currentDoc in positional_index[tokens[index]]:
      if currentWordPos+1 in positional_index[tokens[index]][currentDoc]:
        fn(index+1,tokens,currentWordPos+1,currentDoc)
    else:
      return 

#Processing Queries
for i in range(0,n):
  tokens=tokensInp[i]
  wordPosition=0
  currentDoc=""
  for doc in positional_index[tokens[0]]:
    for position in positional_index[tokens[0]][doc]:
      fn(1,tokens,position,doc)

  print("\nNumber of documents retrieved :", len(docs))
  if len(docs):
    print("Names of the documents retrieved : ",end="")
    for i in docs:
      print(i,end=", ")
  else:
    print("Names of the documents retrieved: None")
  docs=set()


Number of documents retrieved : 1
Names of the documents retrieved : file22.txt, 
Number of documents retrieved : 2
Names of the documents retrieved : file32.txt, file472.txt, 